# GTSAM 4.0

## Install

First install all dependencies.


```bash
sudo apt-get install cmake libboost-all-dev libtbb-dev git python-dev
```


Clone the repository into your checkout folder.

```bash
git clone https://bitbucket.org/gtborg/gtsam.git
cd gtsam
git checkout 5a8bd5a # feature/cython_wrapper
```

Then create a build folder and execute cmake.

```bash
mkdir build
mkdir cython_build
cd build
cmake -DGTSAM_INSTALL_CYTHON_TOOLBOX=ON -DGTSAM_CYTHON_INSTALL_PATH=../cython_build ..
```

Now make the project.

```bash
make -j4
sudo make install
sudo ldconfig
```

Installing globally

```bash
cd ..
cd cython_build
sudo cp -R gtsam /usr/local/lib/python2.7/dist-packages/gtsam
```

## Try it

Now let's try gtsam:

In [1]:
from gtsam import *
from math import *
import numpy as np

def test_Pose2SLAMExample():
        # Assumptions
        #  - All values are axis aligned
        #  - Robot poses are facing along the X axis (horizontal, to the right in images)
        #  - We have full odometry for measurements
        #  - The robot is on a grid, moving 2 meters each step

        # Create graph container and add factors to it
        graph = NonlinearFactorGraph()

        # Add prior
        # gaussian for prior
        priorMean = Pose2(0.0, 0.0, 0.0)  # prior at origin
        priorNoise = noiseModel_Diagonal.Sigmas(np.array([0.3, 0.3, 0.1]))
        # add directly to graph
        graph.add(PriorFactorPose2(1, priorMean, priorNoise))

        # Add odometry
        # general noisemodel for odometry
        odometryNoise = noiseModel_Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
        graph.add(BetweenFactorPose2(
            1, 2, Pose2(2.0, 0.0, 0.0), odometryNoise))
        graph.add(BetweenFactorPose2(
            2, 3, Pose2(2.0, 0.0, pi / 2), odometryNoise))
        graph.add(BetweenFactorPose2(
            3, 4, Pose2(2.0, 0.0, pi / 2), odometryNoise))
        graph.add(BetweenFactorPose2(
            4, 5, Pose2(2.0, 0.0, pi / 2), odometryNoise))

        # Add pose constraint
        model = noiseModel_Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
        graph.add(BetweenFactorPose2(5, 2, Pose2(2.0, 0.0, pi / 2), model))

        # Initialize to noisy points
        initialEstimate = Values()
        initialEstimate.insert(1, Pose2(0.5, 0.0, 0.2))
        initialEstimate.insert(2, Pose2(2.3, 0.1, -0.2))
        initialEstimate.insert(3, Pose2(4.1, 0.1, pi / 2))
        initialEstimate.insert(4, Pose2(4.0, 2.0, pi))
        initialEstimate.insert(5, Pose2(2.1, 2.1, -pi / 2))

        # Optimize using Levenberg-Marquardt optimization with an ordering from
        # colamd
        optimizer = LevenbergMarquardtOptimizer(graph, initialEstimate)
        result = optimizer.optimizeSafely()

        for i in range(result.keys().size()):
            pose = result.atPose2(result.keys().at(i))
            print(pose.x(), pose.y(), pose.theta())

        # Plot Covariance Ellipses
        marginals = Marginals(graph, result)
        P = marginals.marginalCovariance(1)

test_Pose2SLAMExample()

(-3.625860963046766e-13, -1.3599145644354518e-12, -5.283576837143024e-13)
(1.9999999999994762, -3.020846928414637e-12, -7.5454556020218e-13)
(4.000000000653626, -3.9288054987053275e-11, 1.5707963267993925)
(4.000000000188866, 1.9999999999018103, 3.1415926535734795)
(2.0000000000947793, 1.9999999999656077, -1.5707963268164322)


# Uninstalling gtsam (python)

If you need to uninstall gtsam. No idea how to uninstall the c++ stuff, but here is how to uninstall the python stuff.

```bash
sudo rm -rf /usr/local/lib/python2.7/dist-packages/gtsam
```